In [1]:
# Importing the necessary dependency libraries
import xgboost as xgb
from xgboost import XGBClassifier 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc, roc_auc_score
import imblearn
from imblearn.over_sampling import SMOTE

In [2]:
# Define model performance test functions
def performance_xgboost(xtrainup,ytrainup,xtrain,ytrain):
    clf.fit(xtrainup, ytrainup)
    # AUC
    auc_scores = roc_auc_score(ytrain, clf.predict_proba(xtrain)[:,1])
    print('auc = ', "%.3f"%auc_scores)
    # accuracy
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(ytrain, clf.predict(xtrain))*100
    print('accuracy = ', "%.1f"%accuracy)
    print('------------------------')
    # 混淆矩阵
    import sklearn.metrics as sm
    from sklearn.metrics import confusion_matrix
    matrix = confusion_matrix(ytrain, clf.predict(xtrain), labels=None, sample_weight=None)
    print('混淆矩阵为：')
    print(matrix)
    (tn,fp,fn,tp) = matrix.ravel()
    print('tn=',tn)
    print('fp=',fp)
    print('fn=',fn)
    print('tp=',tp)
    print('------------------------')
    sensitivity = (tp/(tp+fn))*100
    specificity = (tn/(fp+tn))*100
    PPV=tp/(tp+fp)*100
    NPV=tn/(fn+tn)*100
    print(f'PPV = {"%.1f"%PPV}\n({tp}/{(tp+fp)})')
    print(f'NPV = {"%.1f"%NPV}\n({tn}/{(fn+tn)})')
    print(f'sensitivity = {"%.1f"%sensitivity}\n({tp}/{(tp+fn)})')
    print(f'specificity = {"%.1f"%specificity}\n({tn}/{(fp+tn)})')

In [3]:
# Reading data
train = pd.read_csv('F:/Onedrive/JIMMY/Python/Notebook/data_set/inuse/train.csv',encoding='gbk')
test = pd.read_csv('F:/Onedrive/JIMMY/Python/Notebook/data_set/inuse/test.csv',encoding='gbk')
validation = pd.read_csv('F:/Onedrive/JIMMY/Python/Notebook/data_set/inuse/validation_new04.csv',encoding='gbk')

xtrain = train.iloc[:,9:14]
xtest = test.iloc[:,9:14]
xv = validation.iloc[:,9:14]
ytrain = train.iloc[:,-1]
ytest = test.iloc[:,-1]
yv = validation.iloc[:,-1]

# Up-sampling processing
sm = SMOTE(sampling_strategy={1: 2000},random_state=100) 
xtrainup,ytrainup = sm.fit_resample(xtrain,ytrain)

C:\Users\jinzh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 1582)
  warnings.warn(


In [4]:
# Model Instantiation
clf = XGBClassifier(n_estimators=400,
                     max_depth=5,
                     min_child_weight=0,
                     gamma=0,
                     subsample=1,
                     colsample_bytree=0.7,
                     reg_alpha=1,
                     learning_rate =0.01,
                     use_label_encoder=False,
                     objective= 'binary:logistic',
                     tree_method='auto',
                     n_jobs=-1
                    ,scale_pos_weight=1) 
clf.fit(xtrainup, ytrainup)

[16:34:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=0, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='auto', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [5]:
# Testing the model on the training set
performance_xgboost(xtrainup,ytrainup,xtrain,ytrain)

[16:34:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
auc =  0.916
accuracy =  83.5
------------------------
混淆矩阵为：
[[1270  312]
 [  45  541]]
tn= 1270
fp= 312
fn= 45
tp= 541
------------------------
PPV = 63.4
(541/853)
NPV = 96.6
(1270/1315)
sensitivity = 92.3
(541/586)
specificity = 80.3
(1270/1582)


In [6]:
# Testing the model on the internal validation set
performance_xgboost(xtrainup,ytrainup,xtest,ytest)

[16:34:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
auc =  0.899
accuracy =  82.7
------------------------
混淆矩阵为：
[[531 135]
 [ 26 238]]
tn= 531
fp= 135
fn= 26
tp= 238
------------------------
PPV = 63.8
(238/373)
NPV = 95.3
(531/557)
sensitivity = 90.2
(238/264)
specificity = 79.7
(531/666)


In [7]:
# Testing the model on the external validation set
performance_xgboost(xtrainup,ytrainup,xv,yv)

[16:34:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
auc =  0.883
accuracy =  72.8
------------------------
混淆矩阵为：
[[415 217]
 [ 19 216]]
tn= 415
fp= 217
fn= 19
tp= 216
------------------------
PPV = 49.9
(216/433)
NPV = 95.6
(415/434)
sensitivity = 91.9
(216/235)
specificity = 65.7
(415/632)
